# SLSTR Composite Plotter

    Version: 1.0
    Date:    27/09/2019
    Author:  Ben Loveday (Plymouth Marine Laboratory) and Hayley Evers-King (EUMETSAT)
    Credit:  This code was developed for EUMETSAT under contracts for the Copernicus 
             programme.
    License: This code is offered as open source and free-to-use in the public domain, 
             with no warranty.

**What is this notebook for?**

This notebook will plot a Sentinel-3 SLSTR SST composite.

**What specific tools does this notebook use?**

None

***

In [ ]:
import os
import shutil
import sys
import warnings
import datetime
import numpy as np
import fnmatch
import logging
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import gridspec
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patheffects as path_effects
import xmltodict

warnings.filterwarnings("ignore")

def main(nc_files):

    # cartopy land layers
    land_resolution = '50m'
    land_poly = cfeature.NaturalEarthFeature('physical', 'land', land_resolution,
                                             edgecolor='k',
                                             facecolor=cfeature.COLORS['land'])

    fsz=20

    #plot data
    fig  = plt.figure(figsize=(20,16), dpi=150)
    plt.rc('font', size=fsz)
    gs = gridspec.GridSpec(3, 1, height_ratios=[20,0.5,1])
    gs.update(wspace=0.01, hspace=0.01)

    # set my vertical plotting order and fontsize
    zordcoast=0

    # plot the data
    #vmin=int(np.nanmean(SST_C)-3*np.nanstd(SST_C))-1
    #vmax=int(np.nanmean(SST_C)+3*np.nanstd(SST_C))+1
    #SST_C[SST_C<vmin] = np.nan
    #SST_C[SST_C>vmax] = np.nan

    m = plt.subplot(gs[0,0], projection=ccrs.PlateCarree())
    m.set_extent([-160, -116, 10, 45], ccrs.PlateCarree())

	# read data
    for nc_file in nc_files:
        nc_fid = xr.open_dataset(nc_file)
        LON  = nc_fid.lon
        LAT  = nc_fid.lat
        SST_raw = np.squeeze(nc_fid.sea_surface_temperature.data)
        SST_BIAS = np.squeeze(nc_fid.sses_bias.data)
        SST_C = SST_raw + SST_BIAS - 273.15
        QUALITY_LEVEL = np.squeeze(nc_fid.quality_level.data)
        nc_fid.close()

        # flag data
        SST_C[QUALITY_LEVEL<=4]=np.nan

        # quick subset
        SST_C = SST_C[11220:17400,:]
        LON = LON[11220:17400,:]
        LAT = LAT[11220:17400,:]

        xml_file = os.path.dirname(nc_file)+'/xfdumanifest.xml'
        with open(xml_file) as fd:
            doc = xmltodict.parse(fd.read())
            coords = doc['xfdu:XFDU']['metadataSection']['metadataObject'][2]['metadataWrap']['xmlData']['sentinel-safe:frameSet']['sentinel-safe:footPrint']['gml:posList']

        # split the coords
        lats_lons = coords.split(' ')
        lats = lats_lons[::2]
        lons = lats_lons[1::2]

        lats = np.asarray(lats).astype(float)
        lons = np.asarray(lons).astype(float)

        p1 = m.pcolormesh(LON,LAT,SST_C,cmap=plt.cm.RdYlBu_r,vmin=10,vmax=28,zorder=-1)
        if 'S3B' in nc_file:
            plot_col = 'b'
        else:
            plot_col = 'k'

        p2 = m.plot(lons, lats, c=plot_col, linewidth=1, linestyle='--', zorder=5, alpha=0.5, transform=ccrs.PlateCarree())

    # add embelishments
    m.coastlines(resolution=land_resolution, color='black', linewidth=1)
    m.add_feature(land_poly)
    g1 = m.gridlines(draw_labels = True)
    g1.xlabels_top = False
    g1.ylabels_right = False

    g1.xlabel_style = {'size': fsz, 'color': 'gray'}
    g1.ylabel_style = {'size': fsz, 'color': 'gray'}

    label='Sentinel-3A SLSTR SST\n07/09/2019 (22:27 local time)'    
    txt = plt.annotate(label, xy=(0.66, 1.01), xycoords='axes fraction', size=fsz/1.25,
                   color='k', zorder=100, annotation_clip=False)

    label='Sentinel-3A SLSTR SST\n08/09/2019 (00:08 local time)'    
    txt = plt.annotate(label, xy=(0.005, 1.01), xycoords='axes fraction', size=fsz/1.25,
                   color='k', zorder=100, annotation_clip=False)

    label='Sentinel-3B SLSTR SST\n08/09/2019 (23:28 local time)'    
    txt = plt.annotate(label, xy=(0.33, 1.01), xycoords='axes fraction', size=fsz/1.25,
                   color='b', zorder=100, annotation_clip=False)

    # add colorbar
    axes0 = plt.subplot(gs[2,0])
    cbar = plt.colorbar(p1, cax=axes0, orientation='horizontal')
    cbar.ax.tick_params(labelsize=fsz) 
    cbar.set_label('SLSTR SST composite [$^{o}$C]',fontsize=fsz)
    fig.savefig('SLSTR_All_SST_California_20190909.png',bbox_inches='tight')

if __name__ == '__main__':
    SLSTR_file1 = '/home/jovyan/work/products/S3A_SL_2_WST____20190908T052719_20190908T070819_20190909T152610_6059_049_076______MAR_O_NT_003.SEN3/20190908052719-MAR-L2P_GHRSST-SSTskin-SLSTRA-20190909152610-v02.0-fv01.0.nc'
    SLSTR_file2 = '/home/jovyan/work/products/S3B_SL_2_WST____20190908T062847_20190908T080946_20190909T153717_6059_029_319______MAR_O_NT_003.SEN3/20190908053709-MAR-L2P_GHRSST-SSTskin-SLSTRB-20190909133350-v02.0-fv01.0.nc'
    SLSTR_file3 = '/home/jovyan/work/products/S3A_SL_2_WST____20190908T070819_20190908T084918_20190909T173018_6059_049_077______MAR_O_NT_003.SEN3/20190908070819-MAR-L2P_GHRSST-SSTskin-SLSTRA-20190909173018-v02.0-fv01.0.nc'

    main([SLSTR_file1, SLSTR_file2, SLSTR_file3])